# Wind Energy Output Prediction- Modelling & Deployment

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#importing the dataset T1.csv
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ae1b71cc049e4c9b85a0caaac9f48f6b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fkN5rfWxqzFUB5jfSeECyUF42hiw9GX3FttH23yskjVe',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ae1b71cc049e4c9b85a0caaac9f48f6b.get_object(Bucket='windenergyprediction-donotdelete-pr-ymmffemp38clbr',Key='T1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.tail()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
50525,31 12 2018 23:10,2963.980957,11.404030,3397.190793,80.502724
50526,31 12 2018 23:20,1684.353027,7.332648,1173.055771,84.062599
50527,31 12 2018 23:30,2201.106934,8.435358,1788.284755,84.742500
50528,31 12 2018 23:40,2515.694092,9.421366,2418.382503,84.297913
50529,31 12 2018 23:50,2820.466064,9.979332,2779.184096,82.274620


In [3]:
#dropping Theoretical power column cause it does'nt contribute to a good prediction
dataset.drop('Theoretical_Power_Curve (KWh)',axis=1,inplace=True)
dataset.dtypes

Date/Time               object
LV ActivePower (kW)    float64
Wind Speed (m/s)       float64
Wind Direction (°)     float64
dtype: object

In [4]:
# Convert Date/Time into Datetime
dataset['Date/Time']=pd.to_datetime(dataset['Date/Time'])

In [5]:
data=dataset.copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 4 columns):
Date/Time              50530 non-null datetime64[ns]
LV ActivePower (kW)    50530 non-null float64
Wind Speed (m/s)       50530 non-null float64
Wind Direction (°)     50530 non-null float64
dtypes: datetime64[ns](1), float64(3)
memory usage: 1.5 MB


In [6]:
data.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Wind Direction (°)
0,2018-01-01 00:00:00,380.047791,5.311336,259.994904
1,2018-01-01 00:10:00,453.769196,5.672167,268.641113
2,2018-01-01 00:20:00,306.376587,5.216037,272.564789
3,2018-01-01 00:30:00,419.645905,5.659674,271.258087
4,2018-01-01 00:40:00,380.650696,5.577941,265.674286


In [7]:
y=data['LV ActivePower (kW)'].values

In [8]:
data.drop('LV ActivePower (kW)',axis=1,inplace=True)
data.head()

,Date/Time,Wind Speed (m/s),Wind Direction (°)
0,2018-01-01 00:00:00,5.311336,259.994904
1,2018-01-01 00:10:00,5.672167,268.641113
2,2018-01-01 00:20:00,5.216037,272.564789
3,2018-01-01 00:30:00,5.659674,271.258087
4,2018-01-01 00:40:00,5.577941,265.674286


In [9]:
print(data.shape)
print(y.shape)

(50530, 3)
(50530,)


In [10]:
#Converting Date/Time into milliseconds
data['Date/Time']=pd.to_numeric(data['Date/Time'])

In [11]:
data.head()

,Date/Time,Wind Speed (m/s),Wind Direction (°)
0,1514764800000000000,5.311336,259.994904
1,1514765400000000000,5.672167,268.641113
2,1514766000000000000,5.216037,272.564789
3,1514766600000000000,5.659674,271.258087
4,1514767200000000000,5.577941,265.674286


In [12]:
data.dtypes

Date/Time               int64
Wind Speed (m/s)      float64
Wind Direction (°)    float64
dtype: object

In [13]:
#splitting the dataset for Training and Testing
X_train,X_test,y_train,y_test=train_test_split(data,y)

## Apllying Random Forest Regressor

In [14]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
regressor=RandomForestRegressor(n_estimators=10,random_state=0)
from sklearn.model_selection import cross_val_score
regressor.fit(X_train,y_train)
train_score = cross_val_score(regressor,X_train,y_train,cv=5)
test_score = cross_val_score(regressor,X_test,y_test,cv=5)

print("train score :",np.mean(train_score))
print("test score :",np.mean(test_score))

train score : 0.9743610127532613
test score : 0.9600978282256151


## Importing Watson Machine Learning API Client

In [16]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-07-11 09:54:29,694 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [17]:
wml_credentials = {
  "apikey": "Yk2xT5XMCo_h0kL0jDBsP-d9Hv-j4gEym1ep2pmT-RhG",
  "iam_apikey_description": "Auto-generated for key c79f1f24-7290-43d1-8e2d-ffa1530c7150",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/f6ed1b7e1a784cfe883690a92e3f8b89::serviceid:ServiceId-72cae3fe-9cbf-4bcb-891f-4d7c2d9f3b42",
  "instance_id": "e5458325-e964-4bd8-9f1b-ac22085beec2",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

## Creating a Client with the ML Service Credentials

In [18]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [19]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME : "Martin",
    client.repository.ModelMetaNames.AUTHOR_EMAIL : "martinpathadan234@gmail.com",
    client.repository.ModelMetaNames.NAME : "Wind_energy_pred"
}

In [20]:
model_artifact = client.repository.store_model(regressor,meta_props = model_props)

In [21]:
model_artifact

{'metadata': {'guid': '6886e8fc-b6e3-4181-8209-315f3b553759',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/published_models/6886e8fc-b6e3-4181-8209-315f3b553759',
  'created_at': '2020-07-11T09:54:30.457Z',
  'modified_at': '2020-07-11T09:54:30.509Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/published_models/6886e8fc-b6e3-4181-8209-315f3b553759/learning_configuration',
  'author': {'name': 'Martin'},
  'name': 'Wind_energy_pred',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/published_models/6886e8fc-b6e3-4181-8209-315f3b553759/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/published_models/6886e8fc-b6e3-4181-8209-315f3b553759/feedback',
  'latest_version': {'url': '

In [22]:
guid = client.repository.get_model_uid(model_artifact)
guid

'6886e8fc-b6e3-4181-8209-315f3b553759'

## Deployment Creation

In [23]:
deploy = client.deployments.create(guid,name = "Wind_energy_pred_d")



#######################################################################################

Synchronous deployment creation for uid: '6886e8fc-b6e3-4181-8209-315f3b553759' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6fdc4db7-8094-4435-92a3-ef9a9264ac3d'
------------------------------------------------------------------------------------------------




In [29]:
#List of deployments
client.deployments.list()

------------------------------------  ------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
6fdc4db7-8094-4435-92a3-ef9a9264ac3d  Wind_energy_pred_d  online  DEPLOY_SUCCESS  2020-07-11T09:54:41.093Z  scikit-learn-0.20  model
------------------------------------  ------------------  ------  --------------  ------------------------  -----------------  -------------


In [25]:
deploy

{'metadata': {'guid': '6fdc4db7-8094-4435-92a3-ef9a9264ac3d',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/deployments/6fdc4db7-8094-4435-92a3-ef9a9264ac3d',
  'created_at': '2020-07-11T09:54:41.093Z',
  'modified_at': '2020-07-11T09:54:41.451Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'Wind_energy_pred_d',
  'scoring_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/deployments/6fdc4db7-8094-4435-92a3-ef9a9264ac3d/online',
  'deployable_asset': {'name': 'Wind_energy_pred',
   'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/published_models/6886e8fc-b6e3-4181-8209-315f3b553759',
   'guid': '6886e8fc-b6e3-4181-8209-315f3b553759',
   'created_at': '2020-07-11T09:54:41.064Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'scikit-learn-0.20',
  'stat

## Scoring endpoint url

In [26]:
scoring_url = client.deployments.get_scoring_url(deploy)

In [27]:
scoring_url

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/e5458325-e964-4bd8-9f1b-ac22085beec2/deployments/6fdc4db7-8094-4435-92a3-ef9a9264ac3d/online'